# Get a& filter nrc words

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

tf_anger = pd.read_csv('/content/drive/MyDrive/tf-idf/Proper/tfList/tf_anger')
tf_disgust = pd.read_csv('/content/drive/MyDrive/tf-idf/Proper/tfList/tf_disgust')
tf_fear  = pd.read_csv('/content/drive/MyDrive/tf-idf/Proper/tfList/tf_fear')
tf_joy = pd.read_csv('/content/drive/MyDrive/tf-idf/Proper/tfList/tf_joy')
tf_sadness = pd.read_csv('/content/drive/MyDrive/tf-idf/Proper/tfList/tf_sadness')

len(tf_anger), len(tf_joy)

def corr(df):
    temp = 0
    temp2  = []
    #I've deleted 'away', and 'no'
    newList = ['you', 'we', 'they', 'them', 'us', 'him', 'her', 'me', 'the', 'an', "'s", 'it', 'he', 'she', 'said', 'ah',
           'but', 'what', 'where', 'who', 'why', 'when', 'then', 'shall', 'and', 'that',
           'there', 'this', 'these', 'those', 'am', 'is', 'was', 'were', 'mr', 'Mr', 'thou', 'oh']
    for i in range(len(df['index'])):
        if len(df['index'][i]) == 1:
            temp+=1
            temp2.append(i)
        if df['index'][i] in newList:
            temp2.append(i)
            temp+=1
    corrdf = df.drop(temp2, axis=0)
    corrdf.reset_index(inplace=True)
    corrdf.drop(axis=1, labels=['level_0'], inplace=True)
    return corrdf

tff_anger = corr(tf_anger)
tff_disgust = corr(tf_disgust)
tff_fear = corr(tf_fear)
tff_joy = corr(tf_joy)
tff_sadness = corr(tf_sadness)

nrc_df = pd.read_csv('/content/drive/My Drive/NRC.txt', encoding= 'unicode-escape', sep="\t")
nrc_df.columns = ['Word', 'Emotion', 'Member']

from sklearn.utils import shuffle
def getWord(emotion):

  not_index = nrc_df[nrc_df['Emotion'] != emotion].index
  a = nrc_df.drop(not_index)
  a_1 = a.drop(a[a['Member'] != 1].index)
  myDF = shuffle(a_1, random_state=33)
  myDF = myDF.reset_index().drop(columns='index')
  return myDF

nrc_anger = getWord('anger')
#nrc_anger

nrc_disgust = getWord('disgust')
nrc_fear = getWord('fear')
nrc_joy = getWord('joy')
nrc_sadness = getWord('sadness')

def HiLex(nrc, tf):
    temp = []
    tf = list(tf['index'])
    for w in nrc['Word']:
        if w in tf[:1024]:
            temp.append(w)
    return temp

a = HiLex(nrc_anger, tff_anger)
#a

d = HiLex(nrc_disgust, tff_disgust)
f = HiLex(nrc_fear, tff_fear)
j = HiLex(nrc_joy, tff_joy)
s = HiLex(nrc_sadness, tff_sadness)

l1 = [a,d,f,j,s]
addup = []
for i in l1:
    print(len(i))
    addup.append(len(i))


added=0
for num in addup:
    added+=num
added/len(addup)

# Save words to txt files for future use

In [ ]:
l2 = []
for i in l1:
    l2.append(i)

len(l2)

','.join(l2[0])

filetosave = open('wordlist.txt', 'w')
for i in l2:
    filetosave.write(",".join(i))
    filetosave.write("\n")
filetosave.close()

filetosave2 = open('wordlist2.txt', 'w')
for i in l2:
    filetosave2.write(",".join(i))
    filetosave2.write("\n\n")
filetosave2.close()

# Semantic Sim with model:
## all-mpnet-base-v2

In [ ]:
data_alpha = pd.read_csv('/content/drive/MyDrive/ISEAR_5428/isear_2.csv')

import torch
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

## test QA

query_embedding = model.encode("London's population")
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district',
                                 'Hello, Tehran is the capital of Iran!',
                                  'Britain is a very large kingdom.',
                                  'London is a big metropolice'])

x = util.cos_sim(query_embedding, passage_embedding)
x.shape
print("Similarity:", x)

## Modeling

#Modeling words & converting to tensor
#Modeling words
passage_passage_anger = model.encode(a)
passage_passage_disgust = model.encode(d)
passage_passage_fear = model.encode(f)
passage_passage_joy = model.encode(j)
passage_passage_sadness = model.encode(s)

#transforming numpy.array to torch tensor /START
passage_passage_anger_tch = torch.from_numpy(passage_passage_anger)
passage_passage_disgust_tch = torch.from_numpy(passage_passage_disgust)
passage_passage_fear_tch = torch.from_numpy(passage_passage_fear)
passage_passage_joy_tch = torch.from_numpy(passage_passage_joy)
passage_passage_sadness_tch = torch.from_numpy(passage_passage_sadness)
#transforming numpy.array to torch tensor /END

passage_list = [passage_passage_anger_tch, passage_passage_disgust_tch, passage_passage_fear_tch, passage_passage_joy_tch, passage_passage_sadness_tch]

matrix_01 = []
for memory in data_alpha['SIT']:
  #query_embedded = model.encode(memory)
  query_embedded_tch = torch.from_numpy(model.encode(memory))
  for emoWord in passage_list:
    matrix_01.append(torch.mean(util.cos_sim(query_embedded_tch, emoWord)))

matrix_numpy = []
matrix_01_copy = matrix_01.copy()
for tensor in matrix_01_copy:
  matrix_numpy.append(tensor.numpy())

#Changing the matrix dtype to numpy
matrix_numpy = np.array(matrix_numpy)
type(matrix_numpy)
#Converting 7577*1 to 1*7577
matrix_numpy_1d = np.ravel(matrix_numpy)

#reshaping 1*7577 to 7577*5
matrix_2d_alpha = matrix_numpy_1d.reshape((5428,5))
matrix_2d_alpha

matrix_df = pd.DataFrame(matrix_2d_alpha, columns=['anger', 'disgust', 'fear', 'joy', 'sadness'])
matrix_df['Recorded Emotion'] = data_alpha['Field1']
#matrix_df.to_csv('/content/drive/MyDrive/tf-idf/mpnet-v2/ss1024(corrected).csv', index=False)
matrix_df

## Read the SS matrix


!pip install --upgrade pandas

from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd
matrix_df = pd.read_csv('/content/drive/MyDrive/tf-idf/mpnet-v2/ss1024(corrected).csv')
matrix_df

## Do ML

import numpy as np
matrix_b = matrix_df.copy()
#Run Random Forest
from sklearn.model_selection import train_test_split

features = list(matrix_b.columns)[:-1]
labels = list.pop(list(matrix_b.columns))

#splitting features and lables
X = matrix_b[features]
y = matrix_b[labels]

X_np = np.array(X)
y_np = np.array(y).ravel()

#Creating train/test splits
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import KFold

#function to get metrics from resulting dictionary
#and round them to 3 
def getScores(dict):
  scores = ['test_accuracy', 'test_f1_macro', 'test_precision_macro', 'test_recall_macro']
  temp_ls= []
  for score in scores:
    temp = 0
    temp = round(dict[score].mean(), 3)
    temp_ls.append(temp)
  return temp_ls

clf_LoReg = LogisticRegression(random_state=33).fit(X,y)
clf_LoReg.score(X,y)
LR_cv = cross_validate(clf_LoReg, X_np, y_np, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
w = getScores(LR_cv)
w

## Random Forest

#Training RF
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import KFold
rf_clf = RandomForestClassifier(n_estimators=100)
rf_cv_macro = cross_validate(rf_clf, X_np, y_np, cv=10,scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

x = getScores(rf_cv_macro)
x

## SVM

from sklearn import svm
svm_clf = svm.SVC(kernel='rbf', C=1, random_state=42)
svm_cv = cross_validate(svm_clf, X_np, y_np, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
y = getScores(svm_cv)
y

#fitting SVM
svm_clf.fit(X_np, y_np)

#save SVM
import pickle
filename = 'finalized_model2.sav'
pickle.dump(svm_clf, open(filename, 'wb'))

## G-Boosted

from sklearn.ensemble import GradientBoostingClassifier
gb_cv = cross_validate((GradientBoostingClassifier(
                        n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)),X_np, y_np, cv=10,
                        scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
z = getScores(gb_cv)
z

## DF for all metrics

df_1 = pd.DataFrame(index=['Accuracy','F1-score','Precision', 'Recall'], columns=['Random_Forest', 'SVM', 'G-Boosted_Tree'])
df_1['Random_Forest'] = x
df_1['SVM'] = y
df_1['G-Boosted_Tree'] = z
df_1

## PerClass

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import KFold
rf_clf = RandomForestClassifier(n_estimators=100)
from sklearn import svm
svm_clf = svm.SVC(kernel='rbf', C=1, random_state=42)

matrix_b = matrix_df.copy()
#Run Random Forest
from sklearn.model_selection import train_test_split

features = list(matrix_b.columns)[:-1]
labels = list.pop(list(matrix_b.columns))

#splitting features and lables
X = matrix_b[features]
y = matrix_b[labels]

X_np = np.array(X)
y_np = np.array(y).ravel()

#Creating train/test splits
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

#create df
iterables = [["anger", "disgust", "fear", "joy", "sadness"], ["accuracy", "f1-score", "precision", "recall"]]

index = pd.MultiIndex.from_product(iterables, names=["Emotion", "Metrics"])

df_2 = pd.DataFrame(index = index, columns=['(Random_Forest)', '(SVM)', '(G-Boosted_Tree)', '(NaiveBayes_GaussianNB)'])

y_pred_gb = cross_val_predict(GradientBoostingClassifier(n_estimators=100, learning_rate=0.9, max_depth=1, random_state=0), X_np, y_np, cv=10)
gb_class_report = classification_report(y_np, y_pred_gb, output_dict=True)

y_pred_nbg = cross_val_predict(GaussianNB(), X_np, y_np, cv=10)
nbg_class_report = classification_report(y_np, y_pred_nbg, output_dict=True)

y_pred_rf = cross_val_predict(rf_clf, X_np, y_np, cv=10)
rf_class_report = classification_report(y_np, y_pred_rf, output_dict=True)

y_pred_svm = cross_val_predict(svm_clf, X_np, y_np, cv=10)
svm_class_report = classification_report(y_np, y_pred_svm,output_dict=True)

list1 = ['anger', 'disgust', 'fear', 'joy', 'sadness']
list2 = ['f1-score', 'precision', 'recall']

#FOR RANDOM--FOREST
for i in list1:
  for j in list2:
    df_2['(Random_Forest)'][i][j] = round(rf_class_report[i][j], ndigits=2)
    df_2['(Random_Forest)'][i]['accuracy'] = round(rf_class_report['accuracy'], ndigits=2)

#FOR SVM
for i in list1:
  for j in list2:
    df_2['(SVM)'][i][j] = round(svm_class_report[i][j], ndigits=2)
    df_2['(SVM)'][i]['accuracy'] = round(svm_class_report['accuracy'], ndigits=2)


#(G-Boosted_Tree)
for i in list1:
  for j in list2:
    df_2['(G-Boosted_Tree)'][i][j] = round(gb_class_report[i][j], ndigits=2)
    df_2['(G-Boosted_Tree)'][i]['accuracy'] = round(gb_class_report['accuracy'], ndigits=2)


#(NaiveBayes_GaussianNB)
for i in list1:
  for j in list2:
    df_2['(NaiveBayes_GaussianNB)'][i][j] = round(nbg_class_report[i][j], ndigits=2)
    df_2['(NaiveBayes_GaussianNB)'][i]['accuracy'] = round(nbg_class_report['accuracy'], ndigits=2)

df_2

## Ranking (Maximum)

import numpy as np
list2 = [2,4,2,3,10]
index_min = np.argmax(list2)
index_min

list1 = matrix_df.columns
#treat each row as a list 
matrix_df.iloc[4]

temp = []
for i in range(len(matrix_df.iloc[4])):
    x = matrix_df.iloc[4][i]
    if type(x) != str:
        temp.append(x)
temp

np.argmax(temp)

rank_max = []
temp = []
for i in range(len(matrix_df)):
    temp = []
    for j in range(len(matrix_df.iloc[i])):
        x = matrix_df.iloc[i][j]
        if type(x) != str:
            temp.append(x)
    y = np.argmax(temp)
    rank_max.append(y)
#rank_max

len(rank_max)

labels = matrix_df['Recorded Emotion']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)
le.classes_

true_y = le.transform(labels)
true_y

from sklearn.metrics import precision_recall_fscore_support
y_true = true_y
y_pred = rank_max
#precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0,1,2,3,4])

from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']
print(classification_report(y_true, y_pred, target_names=target_names))

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_true, y_pred, labels=[0,1,2,3,4])
matrix.diagonal()/matrix.sum(axis=1)

matrix

from sklearn.metrics import confusion_matrix
import numpy as np
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4', 'class 5']

#Get the confusion matrix
cm = confusion_matrix(y_true, y_pred)
#array([[1, 0, 0],
#   [1, 0, 0],
#   [0, 1, 2]])

#Now the normalize the diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#array([[1.        , 0.        , 0.        ],
#      [1.        , 0.        , 0.        ],
#      [0.        , 0.33333333, 0.66666667]])

#The diagonal entries are the accuracies of each class
cm.diagonal()
#array([1.        , 0.        , 0.66666667])